This notebook computes several diffs between simulations. Only unique dependencies are considered (i.e., if a package requires another one more than once, only the dependency at the lowest depth is considered). The diffs identify added, removed, common and updated dependencies (1) between consecutive simulations (i.e., through time); (2) between a simulation and the initial simulation (i.e., accumulated delta); (3) between a "latest" release and the "selected" release for each simulation.

In [1]:
import polars as pl

In [2]:
df_all_deps = pl.read_parquet('../data/dependencies.parquet')

print(f'{df_all_deps.n_unique('package')} packages, {df_all_deps.n_unique(['package', 'release', 'date'])} simulations and {len(df_all_deps)} dependencies (avg: {len(df_all_deps) / df_all_deps.n_unique(['package', 'release', 'date']):.2f})')
df_all_deps.sample(10)

3852 packages, 277344 simulations and 11465904 dependencies (avg: 41.34)


package,release,date,path,version,size
str,enum,date,list[str],str,"decimal[38,2]"
"""cirq""","""selected""",2024-05-01,"[""cirq"", ""cirq-web"", … ""numpy""]","""1.23.5""",17715.20
"""transmission-rpc""","""selected""",2023-11-01,"[""transmission-rpc"", ""requests"", ""urllib3""]","""2.0.7""",121.30
"""prefect-gcp""","""selected""",2023-04-01,"[""prefect-gcp"", ""prefect"", ""readchar""]","""4.0.5""",8.30
"""Office365-REST-Python-Client""","""selected""",2023-01-01,"[""office365-rest-python-client"", ""msal"", … ""idna""]","""3.4""",60.10
"""cdk-aurora-globaldatabase""","""latest""",2025-09-01,"[""cdk-aurora-globaldatabase"", ""publication""]","""0.0.3""",7.50
"""dagster-cloud""","""latest""",2023-12-01,"[""dagster-cloud"", ""dagster"", ""pydantic""]","""2.5.2""",372.90
"""types-aiobotocore-lex-models""","""selected""",2024-02-01,"[""types-aiobotocore-lex-models"", ""typing-extensions""]","""4.9.0""",32.00
"""glum""","""latest""",2024-11-01,"[""glum"", ""tabmat"", … ""python-dateutil""]","""2.9.0.post0""",224.50
"""benchmark-runner""","""latest""",2025-09-01,"[""benchmark-runner"", ""azure"", … ""adal""]","""1.2.7""",54.20


Be careful, some packages have their `package` field **distinct from** the first component of their `path` field!

Hereafter, we slightly simplify the dataset, focusing on *unique* dependencies (taking the less nested ones first). 
Note that `depth` starts from `0`. A depth of 0 *always* corresponds to the current package.

In [3]:
df_deps = (
    df_all_deps.lazy()
    # Cache some computation to ease writing polars code
    .with_columns(
        name=pl.col('path').list.get(-1),
        depth=pl.col('path').list.len() - 1,
    )
    # Remove duplicated dependencies, keep the one with the lowest depth
    .group_by('package', 'release', 'date', 'name')
    .agg(
        pl.all().sort_by(pl.col('depth')).first(),
    )
    .select('package', 'release', 'date', pl.col('version').filter(pl.col('depth') == pl.lit(0)).first().over('package', 'release', 'date').alias('release_name'), 'name', 'version', 'depth', 'size')
    .sort('package', 'release', 'date', 'name')
    .collect()
)

df_deps.write_parquet('../data/dependencies_unique.parquet')

print(f'{df_deps.n_unique('package')} packages, {df_deps.n_unique(['package', 'release', 'date'])} simulations and {len(df_deps)} dependencies (avg: {len(df_deps) / df_deps.n_unique(['package', 'release', 'date']):.2f})')
df_deps.sample(10)

3852 packages, 277344 simulations and 3767879 dependencies (avg: 13.59)


package,release,date,release_name,name,version,depth,size
str,enum,date,str,str,str,u32,"decimal[38,2]"
"""qiskit-machine-learning""","""latest""",2024-07-01,"""0.7.2""","""python-dateutil""","""2.9.0.post0""",2,224.50
"""edx-completion""","""latest""",2024-01-01,"""4.4.0""","""click-didyoumean""","""0.3.0""",3,2.70
"""transmission-rpc""","""selected""",2025-02-01,"""3.4.0""","""typing-extensions""","""4.12.2""",1,36.60
"""leafmap""","""latest""",2024-08-01,"""0.36.2""","""ipython""","""8.26.0""",2,798.80
"""google-cloud-monitoring-dashbo…","""latest""",2024-03-01,"""2.15.0""","""urllib3""","""2.2.1""",3,118.20
"""aleph-alpha-client""","""selected""",2023-08-01,"""2.7.1""","""pycparser""","""2.21""",4,115.90
"""valohai-cli""","""selected""",2025-03-01,"""0.22.2""","""gitignorant""","""0.4.0""",1,5.00
"""determined""","""selected""",2025-03-01,"""0.19.9""","""texttable""","""1.7.0""",2,10.50
"""cityseer""","""selected""",2023-10-01,"""3.6.0""","""contourpy""","""1.1.0""",2,237.80


In [4]:
df_diffs = (
    df_deps
    # Get previous simulation
    .pipe(lambda df: df.join(
        other=df.unique('date').sort('date').select(
            pl.col('date'), 
            pl.col('date').shift(1).alias('prev_date'),
            pl.col('date').shift(-1).alias('next_date'),
        ),
        on='date', 
        how='left', 
    ))
    # Identify added, removed, common and changed dependencies
    .pipe(lambda df: df.join(
            other=df,
            left_on=['package', 'prev_date', 'release', 'name'],
            right_on=['package', 'date', 'release', 'name'],
            how='full',
            coalesce=True,
        )
        .select(
            'package', pl.coalesce('date', 'next_date_right'), 'release', 'name', 
            pl.coalesce('depth', 'depth_right'), 
            pl.col('version'),
            pl.col('version_right').alias('other_version'),
            status=pl.when(pl.col('version').is_null()).then(pl.lit('removed'))
                .when(pl.col('version_right').is_null()).then(pl.lit('added'))
                .when(pl.col('version') == pl.col('version_right')).then(pl.lit('common'))
                .otherwise(pl.lit('updated')).cast(pl.Enum(['common', 'updated', 'added', 'removed'])),
        )  
    )
    # Remove first snapshot, since everything is "added" there!
    .filter(pl.col('date') != pl.col('date').min())
    # Remove current package as a dependency
    .filter(pl.col('depth') > 0)
    .sort('package', 'release', 'date', 'name')
)
df_diffs

package,date,release,name,depth,version,other_version,status
str,date,enum,str,u32,str,str,enum
"""2captcha-python""",2023-02-01,"""selected""","""certifi""",2,"""2022.12.7""","""2022.12.7""","""common"""
"""2captcha-python""",2023-02-01,"""selected""","""charset-normalizer""",2,"""3.0.1""","""2.1.1""","""updated"""
"""2captcha-python""",2023-02-01,"""selected""","""idna""",2,"""3.4""","""3.4""","""common"""
"""2captcha-python""",2023-02-01,"""selected""","""requests""",1,"""2.28.2""","""2.28.1""","""updated"""
"""2captcha-python""",2023-02-01,"""selected""","""urllib3""",2,"""1.26.14""","""1.26.13""","""updated"""
…,…,…,…,…,…,…,…
"""zyte-api""",2025-12-01,"""latest""","""tenacity""",1,"""9.1.2""","""9.1.2""","""common"""
"""zyte-api""",2025-12-01,"""latest""","""tqdm""",1,"""4.67.1""","""4.67.1""","""common"""
"""zyte-api""",2025-12-01,"""latest""","""typing-extensions""",3,"""4.15.0""","""4.15.0""","""common"""


In [5]:
df_diffs.write_parquet('../data/deps_diffs.parquet')

Now we do the same, but this time, we compare the "current" simulation with the very first one.

In [6]:
df_diffs_initial = (
    df_deps
    # Get date of first simulation
    .with_columns(
        prev_date=pl.col('date').min()
    )
    .join(
        other=(
             # Let's pretend all simulations have the set of dependencies of the initial one
            df_deps
            .filter(date=pl.col('date').min())
            .select(pl.exclude('date'))
            .join(
                df_deps.unique('date').sort('date').select(
                    pl.col('date'), 
                    pl.col('date').shift(1).alias('prev_date'),
                    pl.col('date').shift(-1).alias('next_date'),
                ), 
                how='cross'
            )
        ),
        left_on=['package', 'prev_date', 'release', 'name'],
        right_on=['package', 'date', 'release', 'name'],
        how='full',
        coalesce=True,
    )
    .select(
        'package', pl.coalesce('date', 'next_date'), 'release', 'name', 
        pl.coalesce('depth', 'depth_right'), 
        pl.col('version'),
        pl.col('version_right').alias('other_version'),
        status=pl.when(pl.col('version').is_null()).then(pl.lit('removed'))
            .when(pl.col('version_right').is_null()).then(pl.lit('added'))
            .when(pl.col('version') == pl.col('version_right')).then(pl.lit('common'))
            .otherwise(pl.lit('updated')).cast(pl.Enum(['common', 'updated', 'added', 'removed'])),
    ) 
    # Remove first snapshot, since everything is "common" there!
    .filter(pl.col('date') != pl.col('date').min())
    # Remove current package as a dependency
    .filter(pl.col('depth') > 0)
    .sort('package', 'release', 'date', 'name')
)
df_diffs_initial

package,date,release,name,depth,version,other_version,status
str,date,enum,str,u32,str,str,enum
"""2captcha-python""",2023-02-01,"""selected""","""certifi""",2,"""2022.12.7""","""2022.12.7""","""common"""
"""2captcha-python""",2023-02-01,"""selected""","""charset-normalizer""",2,"""3.0.1""","""2.1.1""","""updated"""
"""2captcha-python""",2023-02-01,"""selected""","""idna""",2,"""3.4""","""3.4""","""common"""
"""2captcha-python""",2023-02-01,"""selected""","""requests""",1,"""2.28.2""","""2.28.1""","""updated"""
"""2captcha-python""",2023-02-01,"""selected""","""urllib3""",2,"""1.26.14""","""1.26.13""","""updated"""
…,…,…,…,…,…,…,…
"""zyte-api""",2025-12-01,"""latest""","""urllib3""",2,null,"""1.26.13""","""removed"""
"""zyte-api""",2025-12-01,"""latest""","""w3lib""",1,null,"""2.1.1""","""removed"""
"""zyte-api""",2025-12-01,"""latest""","""w3lib""",1,"""2.3.1""","""2.1.1""","""updated"""


In [7]:
df_diffs_initial.write_parquet('../data/deps_diffs_origin.parquet')

"More of the same": let's do this diff again, but this time we'll compare each `latest` release to its corresponding `selected` release.

In the resulting dataframe, **it should be assumed that the missing `release` column always equal to `latest`**. 

In [8]:
df_diffs_latest = (
    df_deps
    .filter(pl.col('release') == 'latest')
    .join(
        other=df_deps.filter(pl.col('release') == 'selected'),
        on=['package', 'date', 'name'],
        how='full',
        coalesce=True,
    )
    .select(
        'package', 'date', 'name', 
        pl.coalesce('depth', 'depth_right'), 
        pl.col('version'),
        pl.col('version_right').alias('other_version'),
        status=pl.when(pl.col('version').is_null()).then(pl.lit('removed'))
            .when(pl.col('version_right').is_null()).then(pl.lit('added'))
            .when(pl.col('version') == pl.col('version_right')).then(pl.lit('common'))
            .otherwise(pl.lit('updated')).cast(pl.Enum(['common', 'updated', 'added', 'removed'])),
    )
    # Remove current package as a dependency
    .filter(pl.col('depth') > 0)
    .sort('package', 'date', 'name')
)
df_diffs_latest

package,date,name,depth,version,other_version,status
str,date,str,u32,str,str,enum
"""2captcha-python""",2023-01-01,"""certifi""",2,"""2022.12.7""","""2022.12.7""","""common"""
"""2captcha-python""",2023-01-01,"""charset-normalizer""",2,"""2.1.1""","""2.1.1""","""common"""
"""2captcha-python""",2023-01-01,"""idna""",2,"""3.4""","""3.4""","""common"""
"""2captcha-python""",2023-01-01,"""requests""",1,"""2.28.1""","""2.28.1""","""common"""
"""2captcha-python""",2023-01-01,"""urllib3""",2,"""1.26.13""","""1.26.13""","""common"""
…,…,…,…,…,…,…
"""zyte-api""",2025-12-01,"""tqdm""",1,"""4.67.1""","""4.67.1""","""common"""
"""zyte-api""",2025-12-01,"""typing-extensions""",3,"""4.15.0""","""4.15.0""","""common"""
"""zyte-api""",2025-12-01,"""urllib3""",2,null,"""2.5.0""","""removed"""


In [9]:
df_diffs_latest.write_parquet('../data/deps_diffs_latest.parquet')